<img src="botfather.png" style="height: 80vh">

Telegram (i moltes altres plataformes) ens ofereixen la possibilitat de crear **bots**, és a dir algorismes que són capaços d'automatizatzar tasques i, en aquest cas, de simular una conversa real amb un objectiu concret.

Cada cop més els bots són més utilitzats, vegeu per exemple l'empresa [correYvuela](http://www.correyvuela.com/) que els fa servir per vendre bitllets d'avió de forma ràpida i eficient.

En aquesta pràctica, i seguint la temàtica de l'anterior, farem un bot que ens serveixi per tenir la llista de la compra interactiva. El bot recordarà què li hem dit que volem comprar i anirà tatxant aquells elements que ja haguem comprat (quan li diguem, és clar!)

# Introducció a Python 3 i AsyncIO

Amb Python 3.5 s'introdueix a Python un estil de programació pseudo-paral·lel, on diferents tasques s'executen de forma *simultània*. Vegem un exemple per entendre-ho millor

In [ ]:
import time

def func1_sync():
    for i in range(5):
        time.sleep(1)
        print('func_1 {}'.format(i))

def func2_sync():
    for i in range(5):
        time.sleep(1)
        print('func_2 {}'.format(i))
        
def main():
    func1_sync()
    func2_sync()
    
main()

És fàcil de veure, simplement mirant el codi, que s'executarà seqüencialment primer la funció 1 i després la 2. Imagineu que ara, aquestes funcions 1 i 2, són part d'un entorn web i necessiten accedir a una base de dades per mostrar-la, simulat en el codi amb un `time.sleep(1)`... deixaríem tots els usuaris penjats fins que no acabéssim de processar.

Existien formes de fer que el procés no bloquegés, però eren poc pràctiques i en ocacions es refiaven de petits *hacks* a les llibreries estàndar.

Mireu ara el següent exemple:

In [ ]:
import asyncio as aio


async def func1_async():
    for i in range(5):
        await aio.sleep(1)
        print('func_1 {}'.format(i))
        
async def func2_async():
    for i in range(5):
        await aio.sleep(1)
        print('func_2 {}'.format(i))
        
async def main():
    # Posem a la llista de tasques les dues funcions
    future1 = aio.ensure_future(func1_async())
    future2 = aio.ensure_future(func2_async())
    
    # Esperem a que totes dues acabin
    await aio.wait([future1, future2])
    
loop = aio.get_event_loop()
loop.run_until_complete(main())

Ara, tal i com podeu provar, els missatges surten intercalats. Quan una de les tasques `async` entra en un estat de bloqueig (per exemple, consultar una base de dades, llegir un fitxer, etc.), es reemprén l'execució d'alguna altra tasca `async` que no estigui bloquejada.

D'aquesta forma, conseguirem poder atendre a diversos usuaris a la vegada de forma *paral·lela* i sense fer-los esperar. És important entendre que realment **no** s'estan fent totes dues tasques a la vegada, sinó que quan una bloqueja es passa a l'altre!

**Tingueu en compte que per la pràctica realment no és necessari crear cap nova funció a més de les que us donem ja fetes, i de fet no caldrà ni cridar-les en moltes ocasions.** Tant els exemples anteriors com les explicacions tenen l'objectiu d'introduir-vos les noves funcionalitats de Python 3.5 i donar-vos la oportunitat de fer-les servir, si així ho voleu.

Per poder fer servir `async` i `await` s'han de seguir algunes normes:

* Una funció `def async` es pot cridar de dues formes diferents, segons si la funció d'on es crida és o no `async`:
  * `async def` $\rightarrow$ `async def`: Caldrà posar `await` abans de la crida, per exemple en el codi superior les crides a `func1_async` i `func2_async`
  * `def` a `async def`: Caldrà fer-ho at través d'una funció de la llibreria `AsyncIO`, les més comuns són:
    * Si volem el retorn de la funció `ensure_future`
    * Si no ens interesa el retorn i simplement volem que executi `get_event_loop().create_task`
    * Si volem espera bloquejant a que acabi `run_until_complete` 
    
* Les funcions normals de python (`def`) es poden cridar de forma normal des de qualsevol tipus de funció, tant `def` com `async def`.

# Bot

## Abans de començar

És imprescindible que, al menys 1 persona de cada parella, tingui Telegram. 

És igual si és:

* Telegram web: https://web.telegram.org
* El client per PC/Mac: https://telegram.org/apps
* El mòbil

Per comoditat d'anar provant, us recomenem el client de PC/Mac

## Creant el bot

Primer de tot heu d'afegir al bot anomenat **@BotFather**. Ho podeu fer directament des del següent enllaç:

https://telegram.me/botfather

El procés és força intuitiu i directe, però si us perdeu seguiu les instruccions a: 

https://core.telegram.org/bots#6-botfather

Obtindreu una clau per poder operar un bot. Tota la pàgina és interesant, val la pena llegir-la en un altre moment per informar-se de totes les possibilitats dels bots!

Per aquesta pràctica, i per l'autocorrector, assegureu-vos d'escriure la clau dins de l'arxiu `TOKEN` que es troba en aquesta mateixa carpeta.

## Codi base del bot

Durant aquesta pràctica utilitzarem la llibreria **telepot** per ajudar-nos en el procés, d'altra banda seria molt llarg de fer i implicaria treballar a massa baix nivell pel que volem fer en aquesta pràctica.

L'objectiu, com ja s'ha dit, és tenir una llista de la compra interactiva, però abans de començar hem d'entendre com funciona la llibreria telepot i el flux d'utilització del bot. Primer de tot, executeu el codi que teniu a continuació i intenteu parlar amb el bot mitjançant el vostre Telegram, observeu que passa!

In [ ]:
import asyncio as aio

import telepot
from telepot.aio.loop import MessageLoop
from telepot.aio.delegate import pave_event_space, per_chat_id, \
    create_open, include_callback_query_chat_id


class ShoppingBot(object):
    """
    Classe principal del bot, configura els clients i inicia
    el bucle per rebre i enviar missatges
    """
    
    def __init__(self):
        """
        Constructor de la classe, inicialitza el bot
        """
        self.bot = None
        self.loop = aio.get_event_loop()
        
    def start(self, token):
        """
        Inicia el bucle per rebre i enviar missatges. Bloqueja per
        complet fins que no s'acaba d'executar
        
        :param token: Token del bot per conenctar a l'API de telegram
        """
        self.bot = telepot.aio.DelegatorBot(token, [
            include_callback_query_chat_id(
                pave_event_space())(
                per_chat_id(), create_open, ShoppingUser, timeout=10),
            ])
        
        self.loop.create_task(MessageLoop(self.bot).run_forever())
        self.loop.run_forever()

Segurament és el primer cop que veieu classes en Python, però com podreu comprovar són molt intuitives. 

* S'assemblen a les de Java/C++ en que `self` actua com a `this`

* Els mètodes es defineixen com si fossin funcions normals de Python. Tenen un primer paràmetre obligatori (i especial), el `self` del punt anterior. 

* El mètode `__init__` és el constructor, s'invoca quan es crea un nou objecte de la classe

* Un objecte es crea de forma directe (`obj = Objecte()`), i per cridar-ne un mètode directament fem `objecte.funcio()`, com en Java, **sense** passar (o ignorant) el pàrametre `self`, que Python gestiona automàticament.

In [ ]:
class ShoppingUser(telepot.aio.helper.ChatHandler):
    """
    Classe per instanciar cada usuari i gestionar-ne els
    missatges.
    """
    
    def __init__(self, *args, **kwargs):
        """
        Constructor de la classe, amb l'únic fi de mostrar un missatge
        per pantalla indicant que s'ha creat l'usuari
        """
        super(ShoppingUser, self).__init__(*args, **kwargs)
        print('Created {}'.format(self.id))
       
    
    async def on_chat_message(self, msg):
        """
        Funció que Telepot cridarà de forma automàtica quan un usuari
        enviï un missatge
        
        :param msg: Objecte que conté, d'entre altres, el missatge que
            l'usuari ha enviat
        """
        content_type, chat_type, chat_id = telepot.glance(msg)
        if 'text' in msg:
            await self.sender.sendMessage(msg['text'])
            print('From {}: {}'.format(self.id, msg['text']))
            
    async def on_close(self, ex):
        """
        Passats 10 segons (o els configurats) d'inactivitat de l'usuari,
        aquest s'elimina de la memòria. Abans però, Telepot crida 
        automàticament aquesta funció per informar-nos i, si cal, poder
        fer quelcom
        
        :param ex: Motiu pel qual es tanca l'usuari, normalment timeout
        """
        print('Closed {}'.format(self.id))

In [ ]:
if __name__ == '__main__':
    # Es crea un bot i inicia
    bot = ShoppingBot()
    bot.start(open('TOKEN').read().strip())

Podeu parar el funcionament mitjançant el botó `stop` de la barra superior, doncs el mètode `start` bloqueja el funcionament.

Detalls a tenir en compte i que haurieu d'haver observat!

* Quan algú escriu per primera vegada al bot (o després d'haver-se tancat), es crea un nou `ShoppingUser`, podeu comprovar-ho amb el missatge per pantalla. A més, cada usuari té un identificador únic, MAI canvia.

* Quan envia un missatge, es crida automàticament la funció `on_chat_message`, i el missatge es troba dins de `msg['text']`.

* Al cap de 10 segons d'inactivitat, l'objecte `ShoppingUser` s'elimina per complet. Si torna a enviar un missatge, es crea de nou, però de 0!

**Abans de passar al codi que heu de programar, us planteja algun problema aquest funcionament??**